In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Import the data
df = pd.read_csv('datasets/house_prices.csv')
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


## BASIC INFO ON THE DATA

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13320 entries, 0 to 13319
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13320 non-null  object 
 1   availability  13320 non-null  object 
 2   location      13319 non-null  object 
 3   size          13304 non-null  object 
 4   society       7818 non-null   object 
 5   total_sqft    13320 non-null  object 
 6   bath          13247 non-null  float64
 7   balcony       12711 non-null  float64
 8   price         13320 non-null  float64
dtypes: float64(3), object(6)
memory usage: 936.7+ KB


In [4]:
df.describe(include='all')

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
count,13320,13320,13319,13304,7818,13320,13247.000000,12711.000000,13320.000000
unique,4,81,1305,31,2688,2117,NaN,NaN,NaN
top,Super built-up Area,Ready To Move,Whitefield,2 BHK,GrrvaGr,1200,NaN,NaN,NaN
freq,8790,10581,540,5199,80,843,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,2.692610,1.584376,112.565627
std,NaN,NaN,NaN,NaN,NaN,NaN,1.341458,0.817263,148.971674
min,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,8.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,1.000000,50.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,2.000000,72.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,2.000000,120.000000


In [5]:
# Which columns have missing data and what's the percentage of the missing data
print('Count of missing data in each column')
print(df.isna().sum())
print()
print()
print('Percentage of missing data in each column')
df.isna().sum() * 100 / 13320

Count of missing data in each column
area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64


Percentage of missing data in each column


area_type        0.000000
availability     0.000000
location         0.007508
size             0.120120
society         41.306306
total_sqft       0.000000
bath             0.548048
balcony          4.572072
price            0.000000
dtype: float64

In [6]:
df.shape

(13320, 9)

** OBSERVATIONS **

- The dataset has 13,320 rows and 9 columns
- 3 of the columns are numeric while the rest are string format
- The columns location, size, society, bath and balcony all have missing data with society having the highest

### HANDLING MISSING DATA

In [7]:
# Let's create a pipeline
df1 = df
df1.shape

(13320, 9)

In [8]:
# Let's look at the columns with missing data and fill or drop the columns
# Missing data in location column

df1[df1['location'].isna()]


,area_type,availability,location,size,society,total_sqft,bath,balcony,price
568,Super built-up Area,Ready To Move,NaN,3 BHK,Grare S,1600,3.0,2.0,86.0


In [9]:
# For location, we will fill the missing data with the mode

location_mode = df1['location'].mode()[0]
df1['location'] = df1['location'].fillna(location_mode)


# For the bath and balcony columns, the median will be used to fill the missing data due to outliers
df1['bath'] = df1['bath'].fillna(df1['bath'].median())
df1['balcony'] = df1['balcony'].fillna(df1['balcony'].median())

# The society column will be dropped as 41% of the data is missing. This could be as a result of people not wanting to include the society.
df1 = df1.drop(columns=['society'])

# The rows having null values for the size column will be dropped, as we cannot impute random figures for the size.
df1 = df1.dropna()

df1.isna().sum()

area_type       0
availability    0
location        0
size            0
total_sqft      0
bath            0
balcony         0
price           0
dtype: int64

### DATA INCONSISTENCIES

- The next step is to fix data inconsistencies such as spaces, spellings, cases and in the case of the size column, not using thesame metric (BHK or RK or Bedrooms only)


In [10]:
# Create a second pipeline
df2 = df1.copy()

In [11]:
#  The size column will be split into 3 columns - Bedroom, Hall and Kitchen looking at the unique values.
def handle_size(house_size):
    size_tokens = house_size.split(' ')
    bedroom = size_tokens[0]
    if 'BHK' == size_tokens[1]:
        hall = 1
        kitchen = 1
    elif 'RK' == size_tokens[1]:
        hall = 0
        kitchen = 1
    else:
        hall = 0
        kitchen = 0
        
    return pd.Series([bedroom, hall, kitchen], index=['bedroom', 'hall', 'kitchen'])

In [12]:
df2[['bedroom', 'hall', 'kitchen']] = df2['size'].apply(handle_size)

# Drop the size column since it is no longer needed
df2 = df2.drop(columns=['size'])

df2.head()

,area_type,availability,location,total_sqft,bath,balcony,price,bedroom,hall,kitchen
0,Super built-up Area,19-Dec,Electronic City Phase II,1056,2.0,1.0,39.07,2,1,1
1,Plot Area,Ready To Move,Chikka Tirupathi,2600,5.0,3.0,120.00,4,0,0
2,Built-up Area,Ready To Move,Uttarahalli,1440,2.0,3.0,62.00,3,1,1
3,Super built-up Area,Ready To Move,Lingadheeranahalli,1521,3.0,1.0,95.00,3,1,1
4,Super built-up Area,Ready To Move,Kothanur,1200,2.0,1.0,51.00,2,1,1


In [13]:
# The columns area_type and location have inconsistent spacing and capitalization (title case will be used)

print('UNIQUE AREA TYPES: ', df2['area_type'].unique())
print('UNIQUE LOCATIONS: ', df2['location'].unique())
print('COUNT OF UNIQUE LOCATIONS: ', df2['location'].nunique())
print()
print()

def fix_spacing_title_case(string):
    return ' '.join(string.split('  ')).title()


df2[['area_type', 'location']] = df2[['area_type', 'location']].map(fix_spacing_title_case)
print('--------------- AFTER REMOVING WHITE SPACES AND FIXING CAPITALIZATION ---------------')
print()
print('UNIQUE AREA TYPES: ', df2['area_type'].unique())
print('UNIQUE LOCATIONS: ', df2['location'].unique())
print('COUNT OF UNIQUE LOCATIONS: ', df2['location'].nunique())

UNIQUE AREA TYPES:  ['Super built-up  Area' 'Plot  Area' 'Built-up  Area' 'Carpet  Area']
UNIQUE LOCATIONS:  ['Electronic City Phase II' 'Chikka Tirupathi' 'Uttarahalli' ...
 '12th cross srinivas nagar banshankari 3rd stage' 'Havanur extension'
 'Abshot Layout']
COUNT OF UNIQUE LOCATIONS:  1304


--------------- AFTER REMOVING WHITE SPACES AND FIXING CAPITALIZATION ---------------

UNIQUE AREA TYPES:  ['Super Built-Up Area' 'Plot Area' 'Built-Up Area' 'Carpet Area']
UNIQUE LOCATIONS:  ['Electronic City Phase Ii' 'Chikka Tirupathi' 'Uttarahalli' ...
 '12Th Cross Srinivas Nagar Banshankari 3Rd Stage' 'Havanur Extension'
 'Abshot Layout']
COUNT OF UNIQUE LOCATIONS:  1293


In [14]:
# It was observed that the column total sqft contains other units such as Sq. Yards and Sq. Meter for 32 records
print('Number of records with a different unit: ', df2[df2['total_sqft'].str.contains(r'[a-zA-Z]', na=False)].shape)
print()

df2[df2['total_sqft'].str.contains(r'[a-zA-Z]', na=False)]


Number of records with a different unit:  (46, 10)



,area_type,availability,location,total_sqft,bath,balcony,price,bedroom,hall,kitchen
410,Super Built-Up Area,Ready To Move,Kengeri,34.46Sq. Meter,1.0,0.0,18.50,1,1,1
648,Built-Up Area,Ready To Move,Arekere,4125Perch,9.0,2.0,265.00,9,0,0
775,Built-Up Area,Ready To Move,Basavanagara,1000Sq. Meter,2.0,1.0,93.00,1,1,1
872,Super Built-Up Area,Ready To Move,Singapura Village,1100Sq. Yards,2.0,2.0,45.00,2,1,1
1019,Plot Area,18-Mar,Marathi Layout,5.31Acres,1.0,0.0,110.00,1,0,0
1086,Plot Area,19-Mar,Narasapura,30Acres,2.0,2.0,29.50,2,0,0
1400,Super Built-Up Area,Ready To Move,Chamrajpet,716Sq. Meter,9.0,1.0,296.00,9,1,1
1712,Plot Area,Ready To Move,Singena Agrahara,1500Sq. Meter,3.0,1.0,95.00,3,0,0
1743,Super Built-Up Area,19-Mar,Hosa Road,142.61Sq. Meter,3.0,1.0,115.00,3,1,1
1821,Plot Area,Ready To Move,Sarjapur,1574Sq. Yards,3.0,1.0,76.00,3,0,0


In [15]:
# Convert the Sq. Yards and Sq. Meter to Sq. Feet
def convert_to_sqft(unit):
    unit = unit.lower()
    if 'yards' in unit:
        return float(unit[0:-9]) * 9
    elif 'meter' in unit:
        return float(unit[0:-9]) * 10.7639
    elif 'perch' in unit:
        return float(unit[0:-5]) * 272.25
    elif 'cents' in unit:
        return float(unit[0:-5]) * 435.6
    elif 'acres' in unit:
        return float(unit[0:-5]) * 43_560
    elif 'guntha' in unit:
        return float(unit[0:-6]) * 1_089
    elif 'grounds' in unit:
        return float(unit[0:-7]) * 2_400
    else:
        return unit


df2['total_sqft'] = df2['total_sqft'].apply(convert_to_sqft)
print('Number of records with a different unit :', df2['total_sqft'].str.contains('Sq').sum())

Number of records with a different unit : 0


In [16]:
# Total sqft should be in float and not an object. The column will be split into the columns sqft_max, sqft_min & sqft_avg

def convert_col_total_sqft(total_sqft):
   
    if isinstance(total_sqft, str) and '-' in total_sqft:
        sqft_min, sqft_max = total_sqft.split('-')
        sqft_min, sqft_max = float(sqft_min), float(sqft_max)
        sqft_avg = (sqft_min + sqft_max) / 2
    else:
        sqft_min = float(total_sqft)
        sqft_max = sqft_min
        sqft_avg = sqft_min
        
    return pd.Series([sqft_min, sqft_max, sqft_avg], index=['sqft_min', 'sqft_max', 'sqft_avg'])

df2[['sqft_min', 'sqft_max', 'sqft_avg']] = df2['total_sqft'].apply(convert_col_total_sqft)
df2 = df2.drop(columns=['total_sqft'])
df2.head(2)

,area_type,availability,location,bath,balcony,price,bedroom,hall,kitchen,sqft_min,sqft_max,sqft_avg
0,Super Built-Up Area,19-Dec,Electronic City Phase Ii,2.0,1.0,39.07,2,1,1,1056.0,1056.0,1056.0
1,Plot Area,Ready To Move,Chikka Tirupathi,5.0,3.0,120.00,4,0,0,2600.0,2600.0,2600.0


In [19]:
#  Looking at the data types of the columns, the column bedroom should be float/int not object
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13304 entries, 0 to 13319
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13304 non-null  object 
 1   availability  13304 non-null  object 
 2   location      13304 non-null  object 
 3   bath          13304 non-null  float64
 4   balcony       13304 non-null  float64
 5   price         13304 non-null  float64
 6   bedroom       13304 non-null  object 
 7   hall          13304 non-null  int64  
 8   kitchen       13304 non-null  int64  
 9   sqft_min      13304 non-null  float64
 10  sqft_max      13304 non-null  float64
 11  sqft_avg      13304 non-null  float64
dtypes: float64(6), int64(2), object(4)
memory usage: 1.3+ MB


In [20]:
df2['bedroom'] = df2['bedroom'].astype('float')
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13304 entries, 0 to 13319
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area_type     13304 non-null  object 
 1   availability  13304 non-null  object 
 2   location      13304 non-null  object 
 3   bath          13304 non-null  float64
 4   balcony       13304 non-null  float64
 5   price         13304 non-null  float64
 6   bedroom       13304 non-null  float64
 7   hall          13304 non-null  int64  
 8   kitchen       13304 non-null  int64  
 9   sqft_min      13304 non-null  float64
 10  sqft_max      13304 non-null  float64
 11  sqft_avg      13304 non-null  float64
dtypes: float64(7), int64(2), object(3)
memory usage: 1.3+ MB


### OUTLIER DETECTION & REMOVAL

- A third pipeline (df3) will be created
- Outliers in the numerical columns (bath, balcony, price, bedroom, hall, kitchen, sqft_min, sqft_max, sqft_avg) will be removed


In [22]:
df3 = df2.copy()
df3.head(2)

,area_type,availability,location,bath,balcony,price,bedroom,hall,kitchen,sqft_min,sqft_max,sqft_avg
0,Super Built-Up Area,19-Dec,Electronic City Phase Ii,2.0,1.0,39.07,2.0,1,1,1056.0,1056.0,1056.0
1,Plot Area,Ready To Move,Chikka Tirupathi,5.0,3.0,120.00,4.0,0,0,2600.0,2600.0,2600.0


In [24]:
def remove_outliers(df, col):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)

    iqr = q3 - q1

    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    return df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]


numerical_columns = ['bath', 'balcony', 'price', 'bedroom', 'hall', 'kitchen', 'sqft_min', 'sqft_max', 'sqft_avg']
for col in numerical_columns:
    df3 = remove_outliers(df3, col)

print('Original dataset shape: ', df2.shape)
print('Shape after removal of outliers: ', df3.shape)
        

Original dataset shape:  (13304, 12)
Shape after removal of outliers:  (9451, 12)


In [25]:
df3.describe()

,bath,balcony,price,bedroom,hall,kitchen,sqft_min,sqft_max,sqft_avg
count,9451.000000,9451.000000,9451.000000,9451.000000,9451.0,9451.0,9451.000000,9451.000000,9451.000000
mean,2.214369,1.616231,67.460826,2.361443,1.0,1.0,1272.796813,1275.290457,1274.043635
std,0.558881,0.736578,30.937325,0.584030,0.0,0.0,312.207258,311.651138,311.645515
min,1.000000,0.000000,10.000000,1.000000,1.0,1.0,480.000000,480.000000,480.000000
25%,2.000000,1.000000,45.000000,2.000000,1.0,1.0,1080.000000,1080.000000,1080.000000
50%,2.000000,2.000000,60.000000,2.000000,1.0,1.0,1230.000000,1232.000000,1231.000000
75%,3.000000,2.000000,83.000000,3.000000,1.0,1.0,1475.000000,1480.000000,1478.000000
max,4.000000,3.000000,190.000000,4.000000,1.0,1.0,2080.000000,2090.000000,2080.000000


# POSSIBLE CONSIDERATIONS FOR FURTHER DATA WRANGLING

- Houses in thesame location with lesser sqft_avg/sqft_min/sqft_max should not have a higher price than a house with a greater sqft_avg/sqft_min/sqft_max
- Houses in thesame location with a higher bedroom count should not have a lesser price than one with a lesser bedroom count